In [1]:
import numpy as np
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from models.detr import DETR
from models.segmentation import DETRsegm
from hubconf import detr_resnet101_panoptic

In [2]:
# class DETRsegm_3D(nn.Module):
#     def __init__(self):
#         detr = detr_resnet101_panoptic()
#         conv

In [3]:
def sort_func(path):
    path_id = int(path.split('/')[-1].split('_')[1])
    return path_id

In [4]:
def show_image_and_label(image, label):
    fig, axs = plt.subplots(nrows=1,ncols=2, squeeze=False,figsize=(12, 12))
    axs[0, 0].imshow(image)
    axs[0, 0].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Image')
    axs[0, 1].imshow(label)
    axs[0, 1].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Label')

In [5]:
class DatasetForSegmentation(Dataset):
    
    def __init__(self, image_paths, label_paths):
        self.image_paths = image_paths
        self.label_paths = label_paths
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, i):
        image_np_array = nib.load(image_paths[i]).get_fdata()
        image_torch_tensor = torch.from_numpy(image_np_array)
        label_np_array = nib.load(image_paths[i]).get_fdata()
        label_torch_tensor = torch.from_numpy(label_np_array)
        return image_torch_tensor, label_np_array

In [6]:
path = '/home/francisco/workspace/ImageCHD_dataset'

In [7]:
image_paths = glob(f'{path}/*image.nii.gz',recursive=True)
label_paths = glob(f'{path}/*label.nii.gz',recursive=True)

In [8]:
image_paths.sort(key=sort_func)
label_paths.sort(key=sort_func)

In [9]:
dset = DatasetForSegmentation(image_paths,label_paths)

In [10]:
inpt, outp = dset[0]

In [27]:
inpt_right_shape = inpt.transpose(2,0).unsqueeze(0).unsqueeze(0).float()

In [28]:
cv_3d = torch.nn.Conv3d(1,1,9)

In [30]:
result = cv_3d(inpt_right_shape[:,:,:9])

In [32]:
inpt_right_shape.shape,result.shape

(torch.Size([1, 1, 221, 512, 512]), torch.Size([1, 1, 1, 504, 504]))

In [21]:
detr = detr_resnet101_panoptic()

/home/francisco/anaconda3/envs/PyTorch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/francisco/anaconda3/envs/PyTorch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
